In [ ]:
# This module provides functions for file and directory manipulation\
import os

# This module provides functions for copying, moving, and deleting files and directories, 
# making it easier to manage file system tasks.
import shutil

# This module allows for various random operations, such as shuffling a list or generating random numbers
import random

# This module provides a variety of image augmentation techniques that can be applied to images 
# to artificially increase the size of a dataset by applying transformations such as rotations, flips and scaling
from imgaug import augmenters as iaa

# This module provides an interface to read and write image data in various formats
import imageio.v2 as imageio

In [ ]:
# Defining the directory path where the face images are stored.
data_dir = 'face_images'

# The os.listdir() function returns a list of entries in the specified directory,
# which in this case are the names of the subdirectories representing different image classes.
class_names = os.listdir(data_dir)

# Creating a dictionary 'class_counts' that maps each class name to the number of images in that class.
class_counts = {class_name: len(os.listdir(os.path.join(data_dir, class_name))) for class_name in class_names}

# The max() function is used to find the highest count of images from the values in the 'class_counts' dictionary.
majority_count = max(class_counts.values())

print(f"Class counts: {class_counts}")

print(f"Majority count: {majority_count}")

Class counts: {'angry': 4953, 'disgust': 547, 'fear': 5121, 'happy': 8989, 'neutral': 6198, 'sad': 6077, 'surprise': 4002}
Majority count: 8989


In [ ]:
# Defining the directory path where the augmented face images will be saved.
output_dir = 'augmented_face_images'

# Checking if the output directory exists; if it doesn't, create it.
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# This augmenter applies a sequence of augmentations to images, enhancing data variability.
augmenter = iaa.Sequential([
    iaa.Fliplr(0.5),  # Applies horizontal flips with a probability of 50%.
    iaa.Affine(rotate=(-20, 20)),  # Rotates images randomly by an angle between -20 and +20 degrees.
    iaa.Crop(percent=(0, 0.1))  # Crops images from each side by a percentage between 0% and 10%.
])

# Listing the subfolders in the data directory, which contain the images for different classes.
subfolders = [os.path.join(data_dir, subfolder) for subfolder in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, subfolder))]

# Setting the maximum count of images to be the majority count (maximum number of images in any subfolder).
max_count = majority_count

# This function reads an image from the specified path, applies augmentations using the augmenter, 
# and returns the augmented image.
def augment_image(image_path):
    image = imageio.imread(image_path)  
    augmented_image = augmenter(image=image)  
    return augmented_image

# Processing each subfolder containing images.
for subfolder in subfolders:
    subfolder_name = os.path.basename(subfolder)  # Extracting the name of the subfolder.
    output_subfolder = os.path.join(output_dir, subfolder_name)  # Creating the corresponding output path.
    
    # Creating the output subfolder if it doesn't already exist.
    if not os.path.exists(output_subfolder):
        os.makedirs(output_subfolder)

    # Listing the images in the current subfolder.
    images = [img for img in os.listdir(subfolder) if os.path.isfile(os.path.join(subfolder, img))]
    image_count = len(images)  
    print(f'Processing {subfolder}: {image_count} images.')

    # Copying existing images from the input subfolder to the output subfolder.
    for img in images:
        shutil.copy(os.path.join(subfolder, img), os.path.join(output_subfolder, img))

    # Checking if the number of images is less than the maximum count.
    if image_count < max_count:
        num_to_augment = max_count - image_count
        print(f'Augmenting {num_to_augment} images in {subfolder}.')

        # Augmenting the required number of images.
        for _ in range(num_to_augment):
            # Randomly selecting an image from the existing images to augment.
            image_to_augment = random.choice(images)
            image_path = os.path.join(subfolder, image_to_augment)

            # Augmenting the selected image.
            augmented_image = augment_image(image_path)

            # Generating a random name for the augmented image.
            augmented_image_name = f"aug_{random.randint(100000, 999999)}.jpg"
            augmented_image_path = os.path.join(output_subfolder, augmented_image_name)

            # Saving the augmented image to the output directory.
            imageio.imwrite(augmented_image_path, augmented_image)

print("Augmentation complete.")

Processing face_images\angry: 4953 images.
Augmenting 4036 images in face_images\angry.
Processing face_images\disgust: 547 images.
Augmenting 8442 images in face_images\disgust.
Processing face_images\fear: 5121 images.
Augmenting 3868 images in face_images\fear.
Processing face_images\happy: 8989 images.
Processing face_images\neutral: 6198 images.
Augmenting 2791 images in face_images\neutral.
Processing face_images\sad: 6077 images.
Augmenting 2912 images in face_images\sad.
Processing face_images\surprise: 4002 images.
Augmenting 4987 images in face_images\surprise.
Augmentation complete.
